In [44]:
import os
import yaml
import json
import ast
import praw
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI

In [4]:
from textwrap import dedent
from reddit_helper import *    

from langchain.llms import OpenAI, Ollama
from langchain_openai import ChatOpenAI

from pydantic import BaseModel

In [5]:
api_file_path = 'api_keys.json'
with open(api_file_path, 'r') as file:
    api_keys = json.load(file)        
openai_gpt35 = ChatOpenAI(model_name="gpt-3.5-turbo", 
                                temperature=0.7,
                                openai_api_key=api_keys['openai'])
openai_gpt4 = ChatOpenAI(model_name="gpt-4", 
                                temperature=0.7,
                                openai_api_key=api_keys['openai'])        
llama31_8b = Ollama(model='llama3.1:8b',)
gemma2_9b = Ollama(model='gemma2:9b',)

In [6]:
cfg_file_path = 'casaai_config.yaml'
with open(cfg_file_path, 'r') as yaml_file:
    cfg = yaml.safe_load(yaml_file)
product_long = cfg.get('product_long_description', '')      
product_short = cfg.get('product_short_description', '')

In [7]:
backstory = "You are a content analyst with expertise in analyzing web content and \
                extracting relevant information. You are responsible for ensuring that \
                content is relevant, high-quality, and aligned with the marketing of \
            {product_short}. "
content_analysis_agent = Agent(
                            role="Content Analyst",
                            goal="Analyze web content and extract relevant information",
                            backstory=backstory,
                            allow_delegation=False,
                            verbose=True,
                            llm=llama31_8b,
                            )

In [8]:
backstory = "You evaluates content by analyzing user interactions, such as likes, \
            shares, comments, and views. You should also consider factors such as \
            user behavior/sentiment."
goal = "Accurately assess the impact and effectiveness of content based on user interactions"
engagement_analysis_agent = Agent(
                            role="Engagement Analyst",
                            goal=goal,
                            backstory=backstory,
                            allow_delegation=False,
                            verbose=True,
                            llm=llama31_8b,
                            )


In [9]:
backstory = "You evaluates content by analyzing its alignment with {product_short}. \
            You should also consider factors such as keyword density, context \
            accuracy, and user intent. You identifies content that effectively \
            meets audience expectations, flags irrelevant material, and \
            provides insights to enhance content targeting."
goal = "Ensure that content is highly pertinent and aligned with the intended topics and audience needs"
relevance_analysis_agent = Agent(
                            role="Relevance Analyst",
                            goal=goal,
                            backstory=backstory,
                            allow_delegation=False,
                            verbose=True,
                            llm=llama31_8b,
                        )

In [10]:
backstory = "Responsible for synthesizing and evaluating the combined outputs from the \
                Content Analysis, Engagement Analysis, and Relevance Analysis agents. You \
                ensures all aspects of content—quality, engagement, and relevance—are \
                harmonized and aligns with the marketing of {product_short}."
goal = "Ensure content is relevant, engaging, and strategically aligned to marketing of product"
content_review_agent = Agent(
                            role="Content Review Supervisor",
                            goal=goal,
                            backstory=backstory,
                            allow_delegation=False,
                            verbose=True,
                            llm=gemma2_9b,
                        )


In [11]:
class ScoreOutput(BaseModel):
    comment_id: str
    score: float
    justification: str

In [12]:
tip_text = "If you do your BEST WORK, I'll give you a $10,000 commission!"

In [46]:
output_format = "[{'comment_id': 'k31npnk', 'relevance_score': 10, 'justification': 'Perfect fit for marketing CasaAI'}, \
                  {'comment_id': 'm23npns', 'relevance_score': 5, 'justification': 'Talking about gardening'},....\
                 ]"

In [82]:
descr = "Analyze posts and associated comments from {input_data} to determine their relevance based on \
identified keywords and phrases w.r.to the marketing of {product_long}. Then provide a score \
out of 10 for each post and comment and provide a justification for each score. {tip_text}"

expected_out = "{output_format} that has comment_id, relevance score, and a brief justification (less than 15 words) \
                explaining the rationale behind the score for each post and its associated comments." 
        
content_analysis_task = Task(
                            description=descr,
                            expected_output=expected_out,
                            agent=content_analysis_agent,
                            )

In [83]:
descr = "Evaluating the level of user interaction with the provided content from {input_data}. This includes\
analyzing metrics such as likes, shares, comments, and views to calculate an overall engagement score out of 10. \
{tip_text}"       

expected_out = "{output_format} that has comment_id, relevance score, and a brief justification (less than 15 words) \
                explaining the rationale behind the score for each post and its associated comments." 

engagement_analysis_task =  Task(
                                description=descr,
                                expected_output=expected_out,
                                agent=engagement_analysis_agent,
                                )

In [84]:
descr = "Assess how well the content in {input_data} aligns with {product_long}. The goal is to assign a \
relevance score out of 10 that reflects the content’s pertinence to its intended audience and its  \
alignment with the product that is marketed. {tip_text}"        

expected_out = "{output_format} that has comment_id, relevance score, and a brief justification (less than 15 words) \
                explaining the rationale behind the score for each post and its associated comments." 
        
relevance_analysis_task = Task(
                                description=descr,
                                expected_output=expected_out,
                                agent=relevance_analysis_agent,
                                )

In [67]:
# descr = "You will get outputs from Content Analysis, Engagement Analysis, and Relevance Analysis agents. \
# You will also get content from {input_data}. You will review the outputs from these agents and the content \
# and provide a final score for each post and comment based on the relevance to the marketing of \
# {product_long}. {tip_text}"        
       
# expected_out = "{output_format} that has comment_id, relevance score, and a brief justification (less than 15 words) \
#                 explaining the rationale behind the score for each post and its associated comments." 
        
# final_scoring_task = Task(
#             description=descr,
#             expected_output=expected_out,
#             agent=content_review_agent,
#         )   


In [68]:
# response_creation_crew = Crew(
#     agents=[content_analysis_agent, engagement_analysis_agent, relevance_analysis_agent,
#             content_review_agent],
#     tasks=[content_analysis_task, engagement_analysis_task, relevance_analysis_task,
#             final_scoring_task],
#     verbose=True,
# )

2024-08-13 11:39:57,333 - 139987221280576 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [86]:
response_creation_crew = Crew(
    agents=[content_analysis_agent, engagement_analysis_agent, relevance_analysis_agent,],
    tasks=[content_analysis_task, engagement_analysis_task, relevance_analysis_task,],
    verbose=True,
    manager_llm=gemma2_9b,
    process=Process.hierarchical,
)

2024-08-13 12:17:32,437 - 139987221280576 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [87]:
reddit_posts, reddit_post_ids = fetch_reddit()
condensed_reddit_data, unique_post_ids, unique_comment_ids = condense_data(reddit_posts, reddit_post_ids)

post_cnt: 24 + 688 = 712
comm_cnt: 0 + 16440 = 16440
cond_cnt: 17128 = 688 + 16440
Cross_ck: 688 = 688


In [95]:
reddit_posts.keys()

dict_keys(['HomeImprovement', 'homeimprovementideas', 'homeimprovement2', 'Sprint', 'InteriorDesignHacks', 'homedecoratingCJ', 'HomeImprovementsitcom', 'AmateurInteriorDesign', 'vr_design', 'homedecorideas', 'DecorHomeIdeas', 'HomeDecorGalore', 'tycoon', 'interiordesignideas', 'InteriorDesignAdvice', 'HomeServices', 'HotwifeXXXCaptions', 'HomeDecorSolution', 'ModernEuroDesign', 'aweism', 'DesignJobs', 'HomeImprovementGuides', 'HomeImprovementSales', 'DesignMyRoom', 'Home', 'InteriorDesignMasters', 'HomeDecorating', 'hireavirtualassistant', 'HomeDecoratingUK', 'HomeImprovement2LTime', 'vr_ar_ux_design', 'Lululemen', 'homedecordeals', 'ScandinavianInterior', 'interiordecorating', 'rupaulsdragrace', 'xxxcaptions', 'HomeDecoratingTips', 'Projectmakeover', 'InteriorDesign', 'HomeImprovementUK', 'realtime_rendering', 'InteriorDesignCanada', 'OnlinePersonalTrainer', 'HomeDecorInspiration', 'VRGaming', 'vfx'])

In [ ]:
reddit_posts

In [94]:
len(reddit_posts['AmateurInteriorDesign']['Virtual Room Redesign'])

6

In [69]:
input_dict = {"input_data": condensed_reddit_data,
              "product_long": product_long,
              "product_short": product_short,
              "tip_text":tip_text,
              "output_format":output_format}

In [70]:
scoring_result = response_creation_crew.kickoff(inputs=input_dict)

 [2024-08-13 11:40:19][DEBUG]: == Working Agent: Content Analyst
 [2024-08-13 11:40:19][INFO]: == Starting Task: Analyze posts and associated comments from [[{'comment_id': '1e1ssz2', 'parent_id': None, 'text': 'Interior design help!', 'author': 'Dazzling_Abrocoma394', 'score': 3}, {'comment_id': 'ld9ak0c', 'parent_id': '1e1ssz2', 'text': "I think you meant carpet in the end. I'd get a huge cream white rug, probably. Do you have some other options already?", 'author': 'andrew_cherniy96', 'score': 1}, {'comment_id': 'ld9n38g', 'parent_id': '1e1ssz2', 'text': 'Due to the colour of the walls i think the paintings are too  "blah" for this space. Id get something more bold/vivid in colour. A real statement design. Then, you could get a luxurious cream/white rug/carpet without everything looking too washed out.', 'author': 'BrushedSpud', 'score': 1}, {'comment_id': 'ldoi9u6', 'parent_id': 'ld9ak0c', 'text': 'Yes you’re right , I meant couch ! I’m definitely leaning towards a beige , off whit

In [71]:
scoring_result.raw

"[{'comment_id': '15n7ljr', 'relevance_score': 10, 'justification': 'Directly promotes CasaAI\\'s design service and offers a discount'}, \n{'comment_id': 'jvkai3b', 'relevance_score': 10, 'justification': 'Highlights CasaAI\\'s service and staff discount, encouraging engagement'}, \n{'comment_id': '1b2d3zx', 'relevance_score': 8, 'justification': 'Addresses a key challenge CasaAI solves:  interior design struggles'}, \n{'comment_id': 'ksnj5ym', 'relevance_score': 6, 'justification': 'Mentions designing spaces, indirectly relevant to CasaAI\\'s purpose'}, \n{'comment_id': '12yfmk4', 'relevance_score': 9, 'justification': 'Discusses laptops used for interior design, potential users of CasaAI'}, \n{'comment_id': 'jhvxbp7', 'relevance_score': 5, 'justification': 'Budget discussion is tangential to CasaAI\\'s value proposition'}, \n{'comment_id': 'jhw5avw', 'relevance_score': 4, 'justification': 'Personal laptop experience unrelated to CasaAI's AI-powered design'}]"

In [81]:
r1 = content_result.raw.split('}]')
r2 = r1[0] + '}]'
r3 = ast.literal_eval(r2)
len(r3), r3

(10,
 [{'comment_id': '1z3m4n5',
   'relevance_score': 10,
   'justification': 'Perfect fit for marketing CasaAI'},
  {'comment_id': '15n7ljr',
   'relevance_score': 9,
   'justification': 'Staff discount code for interior design service fits perfectly'},
  {'comment_id': 'ksnj5ym',
   'relevance_score': 8,
   'justification': 'Talking about struggles in designing a space from scratch'},
  {'comment_id': 'jhw5avw',
   'relevance_score': 7,
   'justification': 'Mentioning budget for laptop to run browser'},
  {'comment_id': '12yfmk4',
   'relevance_score': 6,
   'justification': 'Talking about interior design laptops'},
  {'comment_id': 'jhvxbp7',
   'relevance_score': 5,
   'justification': 'Asking about budget for laptop'},
  {'comment_id': '1b2d3zx',
   'relevance_score': 4,
   'justification': 'Asking about struggles in interior design'},
  {'comment_id': 'k31npnk',
   'relevance_score': 10,
   'justification': 'Perfect fit for marketing CasaAI'},
  {'comment_id': 'm23npns',
   'rel